In [ ]:
from haystack.utils import launch_es
launch_es()

In [ ]:
from qasystem.QASystem import *
from evaluation.xquadevaluation import *
from tqdm.contrib import tzip

In [ ]:
db = Database()
db.connect()
db.delete_all_documents()
# df, dicts = xquad_data_prepare('/home/doinakis/github/haystack/xquad-dataset/xquad.el.json')
# db.add_documents(dicts=dicts)
# qa = QASystem(database=db)


In [ ]:
from scrapping.CNNScrape import *

In [ ]:
docs = scrape_init("./test_db/politiki", "politiki", 0, 2)

In [ ]:
db.add_documents(docs)

In [ ]:
f1_scores = []
em_scores = []
retriever_em = []
actual_ans = []
predicted_ans = []
for question, answer, doc_id in tzip(df.question, df.answer, df.doc_id):
  prediction = qa.pipe.run(
    query=f"{question}", params={"ESRetriever": {"top_k": 3}, "Reader": {"top_k": 1}}
  )
  actual_ans.append(answer['text'])
  predicted_ans.append(prediction['answers'][0].answer)
  retriever_em.append(int(prediction['documents'][0].meta['name'] == doc_id))
  em = compute_em(prediction['answers'][0].answer, answer['text'])
  em_scores.append(em)
  if em == 1:
    f1_scores.append(1)
  else:
    f1_scores.append(compute_f1(prediction['answers'][0].answer, answer['text']))

scores = pd.DataFrame()
scores['actual'] = actual_ans
scores['predicted'] =predicted_ans
scores['em'] = em_scores
scores['f1'] = f1_scores
scores['retriever_em'] = retriever_em

print(f'Exact Match: {scores.em.mean()}')
print(f'F1-Score: {scores.f1.mean()}')
print(f'Accuracy: {scores.retriever_em.mean()}')

In [ ]:
import pandas as pd

In [ ]:
topic_data = pd.read_json('topic_data.json')

In [ ]:
for index, data in topic_data.iterrows():
  data.label = ''.join(data.label)

In [ ]:
to_keep = (topic_data.label == 'Τηλεόραση') | \
  (topic_data.label == 'Τέχνες-Πολιτισμός') | \
  (topic_data.label == 'Επιστολές')         | \
  (topic_data.label == 'Αστυνομικά')        | \
  (topic_data.label == 'Ζώδια')             | \
  (topic_data.label == 'Καιρός')            | \
  (topic_data.label == 'Διαφημίσεις')       | \
  (topic_data.label == 'Βιογραφίες')        | \
  (topic_data.label == 'Κοινωνικά')         | \
  (topic_data.label == 'Συνεντεύξεις')      | \
  (topic_data.label == 'Διάφορα')

In [ ]:
new_topic = topic_data.drop(topic_data[~to_keep].index).reset_index()

In [ ]:
new_topic.label.value_counts()

In [ ]:
other_data_df = pd.DataFrame()
other_data_df['content'] = new_topic.text
other_data_df['url'] = 'no_url'
other_data_df['name'] = 'makedonia'
other_data_df['title'] = 'no_title'
other_data_df['category'] = 'other'

In [ ]:
single =  pd.read_json('/home/doinakis/github/Real-Time-News-Assistant/db/single_file_db.json')
train = pd.read_json('/home/doinakis/github/Real-Time-News-Assistant/db/train_dataset.json')
val = pd.read_json('/home/doinakis/github/Real-Time-News-Assistant/db/val_dataset.json')
test = pd.read_json('/home/doinakis/github/Real-Time-News-Assistant/db/test_dataset.json')

In [ ]:
single['category'].value_counts()

In [ ]:
train['category'].value_counts()

In [ ]:
val['category'].value_counts()

In [ ]:
test['category'].value_counts()

In [ ]:
test

In [ ]:
from haystack.nodes import BM25Retriever
from qasystem.QASystem import *
db = Database()
db.connect()

In [ ]:
bm25 = BM25Retriever(db.document_store)

In [ ]:
documents= bm25.retrieve(query="ποσους ποντους παρεδωσε η αμυνα?", top_k=2)

In [ ]:
documents[0].meta

In [ ]:
for document in documents:
  print(document.meta['name'])

In [ ]:
names = []
for document in documents:
  names.append(document.meta['name'])

In [ ]:
names

In [ ]:
int('document0.jdsson' in names)

In [ ]:
from haystack.nodes import FARMReader

In [ ]:
reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2", max_seq_len=256, doc_stride=128, use_gpu=True, progress_bar=False)

In [ ]:
from evaluation.xquadevaluation import *
df, dicts = xquad_data_prepare('/home/doinakis/github/haystack/xquad-dataset/xquad.el.json')

In [ ]:
from haystack.schema import Document

In [ ]:
doc = Document(content=dicts[0]['content'], id=dicts[0]['meta']['name'])

In [ ]:
prediction = reader.predict(
    query="Ποσους ποντους παρεδωσαν οι Καρολινα Πανθερς?",
    documents=[doc],
    top_k=1
)


In [ ]:
prediction

In [ ]:
prediction['answers'][0].meta

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bm25_scores = [0.9058823529411765, 0.946218487394958, 0.9630252100840336,\
              0.9672268907563025, 0.9739495798319328, 0.9773109243697479,\
              0.980672268907563, 0.9823529411764705, 0.9831932773109243,\
              0.984873949579832]
tfidf_scores = [0.7949579831932773, 0.8722689075630252, 0.9067226890756303,\
                0.9243697478991597, 0.9369747899159664, 0.9436974789915966,\
                0.9453781512605042, 0.9546218487394958, 0.9563025210084034,\
                0.9588235294117647]

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks(range(1,11))
ax.set_ylabel("Accuracy")
ax.set_xlabel("#Documents")
ax.set_title("BM25 Retriever")
ax.grid(True)
ax.plot(range(1,11), bm25_scores)
fig.savefig('bm25-acc.eps', format='eps')


In [ ]:
fig, ax = plt.subplots()
ax.set_xticks(range(1,11))
ax.set_yticks(tfidf_scores)
ax.set_ylabel("Accuracy")
ax.set_xlabel("#Documents")
ax.set_title("TF-IDF Retriever")
ax.grid(True)
ax.plot(range(1,11), tfidf_scores)
# fig.savefig('tf-idf-acc.eps', format='eps')

In [ ]:
# max_seq_len: 256 doc_stride: 128
model_names = ['xlm-roberta-large-squad2', 'triviaqa_bert_el',\
              'squad_bert_el', 'nq_bert_el', \
              'qacombination_bert_el', 'newsqa_bert_el']
model_em_256_128 = [0.5571428571428572, 0.27899159663865547, 0.5714285714285714,\
                    0.38487394957983195, 0.5563025210084034, 0.3915966386554622]
model_f1_256_128 = [0.7576235228024292, 0.40941624672575755, 0.7488355404570559,\
                    0.5751819914035066, 0.7425348937014348, 0.5879300567309794]
model_em_512_256 = [0.5638655462184874, 0.2747899159663866, 0.5722689075630252,\
                    0.38487394957983195, 0.5563025210084034, 0.3915966386554622]
model_f1_512_256 = [0.7682348032816143, 0.40700003996657036, 0.7484068464636124,\
                    0.5766027094375391, 0.7413629981253088, 0.5864217801261221]


In [ ]:
print(round(model_em_256_128,3))